In [1]:
# Importamos las librerias 
import pandas as pd

In [2]:
# Cargamos los datos 
items= pd.read_csv('./Data/items.csv')
reviews= pd.read_csv('./Data/Reviews.csv')
games= pd.read_csv('./Data/Steam_games.csv')
users= pd.read_csv('./Data/Users.csv')

In [3]:
'''En este Endpoint se nos puide devolver Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora'''
def developer(desarrollador : str ):
    # Primero filtramos los datos basandonos en el ID ingresado 
    filter_games=games[games['developer']== desarrollador]
    # Una vez filtrado, agrupamos por año, y hacemos los calculos y creamos las colunmas neceesarias
    filter_games=filter_games.groupby('release_year').agg(Cantidad_de_Items=('app_name', 'size'),
                                             free_games=('price', lambda x: (x == 0.0).sum())).reset_index()
    # Creamos la columna de cantidad de juegos free que hay por año
    filter_games['Contenido Free']= round((filter_games['free_games']/filter_games['Cantidad_de_Items'])*100, 3)
    # Creamos Un diccionario con un ciclo for que itere por cada fila del DataFrame
    # Crea una entrada en el diccionario con un sub-diccionario que contiene la cantidad de items y el contenido free cada año
    dic = {}
    for _, row in filter_games.iterrows():
        año = row['release_year']
        cantidad_items = row['Cantidad_de_Items']
        cont_free = row['Contenido Free']
        
        # Añadir la información al diccionario
        dic[int(año)] = {
            'Cantidad de items': int(cantidad_items),
            'Contenido Free': cont_free
        }
    
    return dic

In [4]:
# Mostramos un ejemplo 
fil= developer('Poppermost Productions')
fil

{2013: {'Cantidad de items': 1, 'Contenido Free': 100.0},
 2018: {'Cantidad de items': 5, 'Contenido Free': 0.0}}

In [5]:
'''Para este endpoint se nos pide devolver cantidad de dinero gastado por el usuario, 
el porcentaje de recomendación en base a reviews.recommend y cantidad de items'''

def userdata( User_id : str ):
    #Seleccionamos las colunas que nos interesan 
    game=games[['id', 'price']]
    # Filtramos la infromacion de los Usuarios, basandose en el usuario ingresado 
    user= users[users['user_id']== User_id]
    # Calculamos el porcentaje de recomendacion de los usuarios 
    por_recommed= round((user['recommend_yes']/(user['recommend_yes']+user['recommend_not']))*100,2).iloc[0]
    # Filtramsos los juegos que tiene ese usuario 
    item_filter= items[items['users_id']==User_id]
    # Hacemos la union de los datos obtenidos
    item_filter= item_filter.merge(game, left_on='item_id', right_on='id', how='inner')
    # Calculamos las estadisticas por usuario, como la cantidad de tiempo jugado y la suma del valor de todos los juegos  
    item_filter= item_filter.groupby('users_id').agg({'playtime_forever':'sum',
                                                      'price':'sum'}).reset_index()
    # Creamos las variables para armar el diccionario y retornarlo
    usuario= item_filter['users_id'].iloc[0]
    tiempo= item_filter['playtime_forever'].iloc[0]
    dinero= item_filter['price'].iloc[0]
    # Se retorna el diccionario 
    return {'Usuario': usuario, 'Dinero invertido': round(dinero,2), 'Tota horas de juego': tiempo,"% de recomendación":por_recommed}

In [6]:
# Mostramos un ejemplo 
fil2= userdata('kube134')
fil2

{'Usuario': 'kube134',
 'Dinero invertido': 5451.87,
 'Tota horas de juego': 80061.0,
 '% de recomendación': 100.0}

In [7]:
'''Para este endpoint se nos pide Debe devolver el usuario que 
acumula más horas jugadas para el género dado y una lista de la acumulación 
de horas jugadas por año de lanzamiento'''

def UserForGenre(genre: str):
    # Selecionamos las columnas que vamos a usar
    item = items[['item_id', 'playtime_forever', 'users_id']]
    game = games[['id', 'principal_genre', 'release_year']]
    # Filtramos los datos basandonos en el genero del juego 
    game = game[game['principal_genre'] == genre]
    # Hacemo un merge con los datos para tener toda la informacion en un solo dataframe
    filter_game = game.merge(item, left_on='id', right_on='item_id', how='inner')
    # Agrupamos y hacemos los calculos
    user_year= filter_game.groupby(['users_id','release_year']).agg({'playtime_forever': 'sum'}).reset_index()
    # Sacamos el usuaro con mas horas jugadas
    max_user= filter_game['users_id'].iloc[0]
    # Filtramos los datos basandonos en el maxio usuario
    user_playtime_by_year = user_year[user_year['users_id'] == max_user][['release_year', 'playtime_forever']]
    # Cambiamos el nombre delas columnas
    user_playtime_by_year.rename(columns={'release_year':'Año','playtime_forever':'Tiempo Jugado'}, inplace=True)
    # Transformamos el dataframe en un diccionario para retornarlo
    year_playtime= user_playtime_by_year.to_dict('records')
    # Hacemo el return 
    return {"Usuario con más horas jugadas para Género " + genre: max_user, "Horas jugadas": year_playtime}
    

In [8]:
fil3=UserForGenre('Indie')
fil3

{'Usuario con más horas jugadas para Género Indie': 'js41637',
 'Horas jugadas': [{'Año': 2005, 'Tiempo Jugado': 0.0},
  {'Año': 2006, 'Tiempo Jugado': 412.0},
  {'Año': 2007, 'Tiempo Jugado': 0.0},
  {'Año': 2008, 'Tiempo Jugado': 435.0},
  {'Año': 2009, 'Tiempo Jugado': 135.0},
  {'Año': 2010, 'Tiempo Jugado': 184.0},
  {'Año': 2011, 'Tiempo Jugado': 270.0},
  {'Año': 2012, 'Tiempo Jugado': 1323.0},
  {'Año': 2013, 'Tiempo Jugado': 1881.0},
  {'Año': 2014, 'Tiempo Jugado': 1073.0},
  {'Año': 2015, 'Tiempo Jugado': 5175.0},
  {'Año': 2016, 'Tiempo Jugado': 1395.0}]}

In [9]:
'''En este endpoint se nos pide Devuelve el top 3 de desarrolladores con juegos MÁS recomendados 
por usuarios para el año dado'''
def best_developer_year( año : int ):
    # Seleccionamos las columnas necesarias
    game=games[['developer','app_name','id','principal_genre','release_year']]
    review=reviews[['item_id','senti_positivo']]
    # Filtramos los datos 
    game=game[game['release_year']== año]
    # Hacemos el merge de la informacion para tenerla junta y operar en ella
    reviews_filter= game.merge(review, left_on='id', right_on='item_id', how= 'inner') 
    # Seleccionamos las columnas con la informacion que necesitemos
    reviews_filter= reviews_filter[['developer','app_name','senti_positivo']]
    # Agrupamos y hacemos los calculos
    reviews_filter=reviews_filter.groupby(['developer']).agg({'senti_positivo':'sum'}).reset_index()
    # Una vez hechos los calculos Ordemamos los datos para sacer el top 3
    reviews_filter= reviews_filter.sort_values(by= 'senti_positivo', ascending= False)
    # Creamos las variables para el diccionario
    developer= list(reviews_filter.iloc[:,0])
    positive=list(reviews_filter.iloc[:,1])
    # Retornamos el diccionario
    return [{'Top 1': [{'Developer':developer[0]},{'Reviews positive':positive[0]}]},
            {'Top 2': [{'Developer':developer[1]},{'Reviews positive':positive[1]}]},
            {'Top 3': [{'Developer':developer[2]},{'Reviews positive':positive[2]}]}]

In [10]:
# Hacemos un ejemplo 
fil4= best_developer_year(2004)
fil4

[{'Top 1': [{'Developer': 'Valve'}, {'Reviews positive': 294}]},
 {'Top 2': [{'Developer': 'The Creative Assembly'}, {'Reviews positive': 10}]},
 {'Top 3': [{'Developer': 'Troika Games'}, {'Reviews positive': 4}]}]

In [11]:
'''Para este endpoint se nos pide Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador 
como llave y una lista con la cantidad total de registros de reseñas de usuarios que se 
encuentren categorizados con un análisis de sentimiento como valor positivo o negativo'''
def developer_reviews_analysis( desarrolladora : str ):
    # Seleccionaos las columnas que vamos a usar
    game= games[['id','developer']]
    review= reviews[['item_id', 'senti_negativo','senti_positivo']]
    # Filtramos los datos por el desarrollador en cuestion 
    game= game[game['developer']== desarrolladora]
    # Hacemos la unidos de los datos para trabajar sobre ellos 
    game_filter= game.merge(review, left_on='id', right_on='item_id', how='inner')
    # Agrupamos y calculamos el total de reviews positivas y negativas por desarrollador
    game_filter= game_filter.groupby('developer').agg({'senti_negativo':'sum',
                                                       'senti_positivo':'sum'}).reset_index()
    # Creamos las variables para el diccionario
    developer= game_filter['developer'].iloc[0]
    review_neg= game_filter['senti_negativo'].iloc[0]
    review_pos= game_filter['senti_positivo'].iloc[0]
    # Retornamos el diccionario 
    return {developer:[{'Positivas': review_pos},{'Negativas':review_neg}]}

In [12]:
# Mostramos un ejemplo
fil5= developer_reviews_analysis('Bethesda Game Studios')
fil5

{'Bethesda Game Studios': [{'Positivas': 228}, {'Negativas': 77}]}

In [13]:
'''Para este endpoint se nos pide armar un sistema de recomendacion un juego ingresado, 
Se arma el sistema de recomendacion implementando el modelo de la similitud del coseno '''

# Importamos las librerias para la implentacion del modelo 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def recomendacion_juego(id_de_producto: int):
    # Encrontramos el juego que se ingreso 
    atributos = games[games['id'] == id_de_producto]
    # Hacemo un condcional para que indique si se encontro el juego 
    if atributos.empty:
        return 'No se encontró un juego con el ID proporcionado, prueba otro ID.'
    
    # Ahora una vez teniendo el juego se filtran los datos basandonos en el genero del juego ingresado 
    juegos = games[games['principal_genre'] == atributos['principal_genre'].iloc[0]]
    # Seleccionamos las colunmas que necesitemos 
    juegos= juegos[['app_name','principal_genre','principal_tag','principal_spec']]
    # Creamo una columna con las caracteristicas de los juegos 
    juegos['caracteristicas'] = juegos[['principal_genre', 'principal_tag', 'principal_spec']].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
    # Seleccionamos los primeros 1000 juegos 
    juegos= juegos[['app_name','caracteristicas']].iloc[:1001,:]
    # inicialozamos el TfidVetorizer 
    vector= TfidfVectorizer()
    # Y le pasamos las columma de caracteristicas de los juegos para entrenarlo
    matriz= vector.fit_transform(juegos['caracteristicas'])
    # Calculamos la matriz de similitud del coseno, que nos da un inidice entre 0 y 1, mas cerca del 1 mas similitud
    matriz_similaridad= cosine_similarity(matriz)
    # Buscamos el indice del juego que ingresamos 
    indice_juego = juegos.index[juegos['app_name'] == atributos['app_name'].iloc[0]].tolist()[0]
    # Enumeramos cada juego con su inidice de similitud 
    similares = list(enumerate(matriz_similaridad[indice_juego]))
    # Ordemanos de mayor a menor 
    similaridades = sorted(similares, key=lambda x: x[1], reverse=True)[1:6]
    # Y creamo una lista con los 5 juegos mas similares al juego que ingresamos 
    juegos_recomendados = [juegos.iloc[i[0]]['app_name'] for i in similaridades]
    # Retornamos la lista de juegos
    return {'Juegos similares que te pueden interesar': juegos_recomendados}
    

In [14]:
# Mostramos un ejemplo
fil6= recomendacion_juego(25830)
fil6

{'Juegos similares que te pueden interesar': ['Ghost Master®',
  'Vegas: Make It Big™',
  'CivCity: Rome',
  'Civilization IV®: Warlords',
  'Railroad Tycoon 3']}

In [15]:
'''Para este endpoint se nos pide algo similiar al anterior, en esta ocasion se pide armar un sistema de recomendacion pero no basado en un juego ingresado, 
sino en un usuario, se arma el sistema de recomendacion implementando el modelo de la similitud del coseno nuevamente '''
def recomendacion_usuario( id_de_usuario: str):
    # Seleccionamos las colunmas a usar 
    items[['item_id','users_id']]
    # Filtramos los datos por el usuario ingresado 
    item= items[items['users_id']==id_de_usuario]
    # Tomo mamos los ID´s de los juegos del usuario 
    ids= item['item_id']
    # Unimos la informacion
    item= item.merge(games, left_on='item_id', right_on='id', how= 'inner')
    # Selecionamos las columnas a usar 
    item= item[['principal_genre','item_id']]
    # hacemos una listo con los generos de los juegos que juega el usuario ingresado 
    genre= pd.DataFrame(item['principal_genre'].value_counts())[:2]
    # hacemos la lista con los generos
    genre= genre.index.tolist()
    # Ahora buscamos los juegos con esos generos para sacar sus caracteristicas
    game= games[games['principal_genre'].isin(genre)][['app_name','id','principal_genre','principal_tag','principal_spec']]
    # De esos juegos sacamos los juegos que el usuario ya tiene y seleccionamos 1000 juegos d la base de datos 
    game= game[~game['id'].isin(ids.tolist())].iloc[:1001,:]
    # Creamoa la columna con las caracteristicas con las caracteristicas de los juegos 
    game['caracteristicas']= game[['principal_genre', 'principal_tag', 'principal_spec']].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
    # Seleccioamos las columnas necesarias
    game=game[['app_name','id','caracteristicas']]
    # Inicamo el TfidVectorizer 
    vector= TfidfVectorizer()
    # Lo entrenamos con las caracteristicas 
    matriz= vector.fit_transform(game['caracteristicas'])
    # Calculamos el coeficiente de similitud 
    matriz_similaridad= cosine_similarity(matriz)
    # Aqui en listamos cada juego con su coeficiente
    '''OBSERVACION, en este caso no enlistamos juegos basandonos en un juego que ingresamos por eso no usaos el indice de juego como en el endpoint anterior,
    por lo que no buscamos juegos que se parezcan a un juego en especial, sino que se parezan entre si para darselos al usuario'''
    similares = list(enumerate(matriz_similaridad))
    # Seleccionamos los 5 juegos 
    similaridades = sorted(similares, key=lambda x: x[0], reverse=True)[1:6]
    # Hacemos una lista con los juegos
    juegos_recomendados = [game.iloc[i[0]]['app_name'] for i in similaridades]
    # Retornamos los juegos 
    return {'Juegos similares que te pueden interesar a ti ' + id_de_usuario: juegos_recomendados}

In [16]:
# Mostramos un ejemplo
fil7= recomendacion_usuario('thequeenpanda')
fil7

{'Juegos similares que te pueden interesar a ti thequeenpanda': ['White Noise Online',
  'Retro Game Crunch',
  'Paradigm Shift',
  'Sweet Lily Dreams',
  'Toribash']}